# Features <a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs-for-neo4j/blob/main/examples/feature_example.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook showcases some of the various features of `yfiles-jupyte-graphs-for-neo4j`. 

For a detailed description for the different mappings, check out the widget [documentation](https://yworks.github.io/yfiles-jupyter-graphs/)


In [ ]:
%pip install yfiles_jupyter_graphs_for_neo4j --quiet
%pip install neo4j --quiet

You can also open this notebook in Google Colab when Google Colab's custom widget manager is enabled:

In [ ]:
try:
  import google.colab
  from google.colab import output
  output.enable_custom_widget_manager()
except:
  pass

<a target="_blank" href="https://colab.research.google.com/github/yWorks/yfiles-jupyter-graphs-for-neo4j/blob/main/examples/feature_example.ipynb"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Connect to the database

In [ ]:
from yfiles_jupyter_graphs_for_neo4j import Neo4jGraphWidget
from neo4j import GraphDatabase

NEO4J_URI      = "neo4j+ssc://demo.neo4jlabs.com" 
NEO4J_USERNAME = "fincen"
NEO4J_PASSWORD = "fincen"
driver = GraphDatabase.driver(uri = NEO4J_URI, auth = (NEO4J_USERNAME, NEO4J_PASSWORD), database = 'fincen')

g = Neo4jGraphWidget(driver)

## Use heat mapping

In [ ]:
max_amount = 120000000
min_amount = 50000

def heat_mapping(element):
    if 'amount' in element['properties']:
        amount = element['properties']['amount']
        normalized_value = (amount - min_amount) / (max_amount - min_amount)
        transformed_value = -1 * (1 - normalized_value) ** 2 + 1
        return max(0, min(1, transformed_value))

g.add_node_configuration('Filing', heat=heat_mapping)

g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 25")

## Visualize geospatial data

In [ ]:
def node_coordinate_mapping(node):
    return (node['properties']['location']['y'], node['properties']['location']['x']) if 'location' in node['properties'].keys() else None

def filing_coordinate_mapping(node):
    return (node['properties']['beneficiary_lat'], (node['properties']['beneficiary_lng'])) if 'beneficiary_lat' in node['properties'].keys() else None

g.add_node_configuration('Country', coordinate=node_coordinate_mapping)
g.add_node_configuration('Entity', coordinate=node_coordinate_mapping)
g.add_node_configuration('Filing', coordinate=filing_coordinate_mapping)

g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 50")

## Configure item visualization

In [ ]:

g.add_relationship_configuration('CONCERNS', thickness_factor= 0.5)
g.add_relationship_configuration('FILED', thickness_factor= 2)
g.del_node_configuration('Entity')
g.add_node_configuration('Filing', size= lambda node:  (55 * (1 + heat_mapping(node)), 55 * (1 + heat_mapping(node)))) 

g.show_cypher("MATCH (s)-[r]->(t) RETURN s,r,t LIMIT 25")